In [ ]:
from copy import deepcopy
from datetime import datetime
from itertools import product
from pathlib import Path

import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
time_now = datetime.now().strftime("%Y-%m-%d")
print(f"Time: {time_now}")

In [ ]:
MODELS = ["PyBoost", "DLinear_NN", "PatchTST_NN", "GPT4TS_NN", "TimesNet_NN", "TimeMixer_NN", "CycleNet_NN"]
DATASETS = ["ILI"]
STRATEGY_TIMES = [
    "FlatWideMIMOStrategy__model_horizon_NaN",
    "MIMOStrategy__model_horizon_NaN",
    "RecursiveStrategy__model_horizon_1.0",
    "RecursiveStrategy__model_horizon_6.0",
]
DATETIMES = ["False", "with_normalization_over_all"]
IDS = ["False", "with_le_normalization_over_all"]
TRANSFORMER_NAMES = [
    "NaN",
    "DifferenceNormalizer",
    "LastKnownNormalizer",
]
TRANSFORMER_REGIMES = ["NaN", "delta"]
TRANSFORMER_TR_FEATURES = ["True"]
TRANSFORMER_TR_TARGETS = ["True"]
REGIMES = ["multivariate", "global"]
CI = ["False", "True"]

In [ ]:
df_path = f"agg_results_{time_now}__normalized_True.csv"

In [ ]:
raw_df = pd.read_csv(df_path)

# Convert bool to object
for col in raw_df.columns:
    if raw_df[col].dtype == bool:
        raw_df[col] = raw_df[col].astype(str)
        print(f"Converted {col} to object")
        
raw_df = raw_df.dropna(subset=['mae_test', 'rmse_test', 'fit_time_test',
       'forecast_time_test', 'mae_val', 'rmse_val', 'fit_time_val', 'forecast_time_val'])
# NaN to "NaN"
raw_df = raw_df.fillna("NaN")

raw_df = raw_df[raw_df["dateteime"].isin(DATETIMES)]
raw_df = raw_df[raw_df["id"].isin(IDS)]

raw_df["tr_features"] = raw_df["tr_features"].str.replace("_metrics_test_v2.csv", "", regex=False)
raw_df["tr_features"] = raw_df["tr_features"].str.replace("_metrics_test_v2_normalized.csv", "", regex=False)

# colname regime -> colname transormer_regime
# colname transformer -> colname transformer_name
# colname tr_target -> colname transformer_tr_target
# colname tr_features -> colname transformer_tr_features
raw_df = raw_df.rename(
    columns={
        "dateteime": "datetime",
        "regime": "transformer_regime",
        "transformer": "transformer_name",
        "tr_target": "transformer_tr_target",
        "tr_features": "transformer_tr_features",
    }
)

# mult ("False" or "True") to regime ("multivariate" or "global")
raw_df["regime"] = np.where(raw_df["mult"] == "False", "global", "multivariate")

# Concatenate strategy_time and model_horizon
raw_df["strategy_time"] = (
    raw_df["strategy_time"] + "__model_horizon_" + raw_df["model_hor"].astype(str)
)

# Drop unnecessary columns
raw_df = raw_df.drop(columns=["model_hor", "hor", "hist", "mult"])
raw_df = raw_df.drop(columns=["dataset", "transformer_tr_target", "transformer_tr_features"])

raw_df["ci"] = raw_df["ci"].replace({"True": "CI", "False": "CM"})
raw_df["mode"] = raw_df["regime"] + " " + raw_df["ci"]
raw_df["mode"] = raw_df["mode"].replace({"global CI": "global", "global CM": "global"})
raw_df.loc[
    (raw_df['model'] == 'PyBoost') & (raw_df['mode'] == 'multivariate CI'),
    'mode'
] = 'multivariate CM'
raw_df = raw_df.drop(columns=["regime", "ci"])

raw_df = raw_df.drop_duplicates(subset=["model", "strategy_time", "datetime", "id", "transformer_name", "mode", "transformer_regime"])

In [ ]:
# Value counts for each column
cols_for_check = [
    col
    for col in raw_df.columns
    if col
    not in [
        "mae_test",
        "rmse_test",
        "mape_test",
        "fit_time_test",
        "forecast_time_test",
        "mae_val",
        "rmse_val",
        "mape_val",
        "fit_time_val",
        "forecast_time_val",
    ]
]

for col in cols_for_check:
    print("Column: ", col)
    print("Type of data: ", raw_df[col].dtype)
    
    print(raw_df[col].value_counts(dropna=False))
    print()

In [ ]:
raw_df.to_csv(f"{df_path[:-4]}_cleaned.csv", index=False)